In [30]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd
import numpy as np
import mlflow

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
acidentes = pd.read_csv("/content/drive/MyDrive/ciencia-de-dados/dataset preprocessado/acidentes.csv",delimiter = ';')

In [33]:
#acidentes=pd.read_csv("acidentes.csv")

In [34]:
acidentes.head()

,DATA,HORA,NATUREZA_ACIDENTE,SITUACAO,BAIRRO,ENDERECO,COMPLEMENTO,BAIRRO_CRUZAMENTO,TIPO,DESCRICAO,AUTO,MOTO,CICLOM,CICLISTA,PEDESTRE,ONIBUS,CAMINHAO,VIATURA,OUTROS,VITIMAS,VITIMASFATAIS,ACIDENTE_VERIFICADO,TEMPO_CLIMA,SITUACAO_SEMAFORO,SINALIZACAO,CONDICAO_VIA,CONSERVACAO_VIA,PONTO_CONTROLE,SITUACAO_PLACA,VELOCIDADE_MAX_VIA,MAO_DIRECAO,DIVISAO_VIA1,PERIODO_DO_DIA,DIA_DA_SEMANA,ANO,MES,QTD_ACIDENTES
0,2015-12-03,11,SEM VITIMA,CANCELADA,CASA AMARELA,AV NORTE,SENTIDO SUBURBIO E/F AO SESC,SEM INFORMACAO,COLISAO,COLISAO SEM VITIMAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,MANHA,QUINTA,2015,12,1
1,2015-06-01,6,SEM VITIMA,FINALIZADA,ENCRUZILHADA,EST DE BELEM,IGREJA BATISTA MEMORIAL DE BELEM,SEM INFORMACAO,COLISAO,SEM VITIMAS,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,MANHA,SEGUNDA,2015,6,1
2,2015-06-01,8,SEM VITIMA,FINALIZADA,BOA VIAGEM,RUA ERNESTO DE PAULA SANTOS,SEMAFORO NO217,SEM INFORMACAO,COLISAO,SEM VITIMA,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,MANHA,SEGUNDA,2015,6,1
3,2015-06-01,8,SEM VITIMA,FINALIZADA,AFOGADOS,AV SUL,AO LADO DA ESTACAO LARGO DA PAZ,SEM INFORMACAO,COLISAO,COLISAO S/V,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,MANHA,SEGUNDA,2015,6,1
4,2015-06-01,8,SEM VITIMA,CANCELADA,MADALENA,RUA BENFICA,EM FRENTE AO CLUBE INTERNACIONAL,SEM INFORMACAO,COLISAO,GM RIBEIRO JA NO LOCAL VT 05,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,MANHA,SEGUNDA,2015,6,1


In [35]:
acidentes['NATUREZA_ACIDENTE'] = acidentes['NATUREZA_ACIDENTE'].replace('VITIMA FATAL', 'COM VITIMA')
acidentes['NATUREZA_ACIDENTE'] = acidentes['NATUREZA_ACIDENTE'].replace('SEM INFORMACAO', np.NaN)
acidentes['NATUREZA_ACIDENTE'] = acidentes['NATUREZA_ACIDENTE'].replace('ENTRADA E SAIDA', np.NaN)
acidentes['NATUREZA_ACIDENTE'] = acidentes['NATUREZA_ACIDENTE'].replace('APOIO', np.NaN)

In [36]:
df = acidentes[['NATUREZA_ACIDENTE','TIPO', 'AUTO', 'MOTO', 'CICLOM', 'CICLISTA','PEDESTRE','ONIBUS','CAMINHAO','VIATURA','TEMPO_CLIMA','SITUACAO_SEMAFORO','SINALIZACAO','CONDICAO_VIA','CONSERVACAO_VIA','PERIODO_DO_DIA']]

In [37]:
df.head()

,NATUREZA_ACIDENTE,TIPO,AUTO,MOTO,CICLOM,CICLISTA,PEDESTRE,ONIBUS,CAMINHAO,VIATURA,TEMPO_CLIMA,SITUACAO_SEMAFORO,SINALIZACAO,CONDICAO_VIA,CONSERVACAO_VIA,PERIODO_DO_DIA
0,SEM VITIMA,COLISAO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,MANHA
1,SEM VITIMA,COLISAO,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,MANHA
2,SEM VITIMA,COLISAO,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,MANHA
3,SEM VITIMA,COLISAO,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,MANHA
4,SEM VITIMA,COLISAO,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,SEM INFORMACAO,MANHA


In [38]:
df['NATUREZA_ACIDENTE'] = df['NATUREZA_ACIDENTE'].replace('SEM INFORMACAO', np.nan)
df['TIPO'] = df['TIPO'].replace('SEM INFORMACAO', np.nan)
df['TEMPO_CLIMA'] = df['TEMPO_CLIMA'].replace('SEM INFORMACAO', np.nan)
df['SINALIZACAO'] = df['SINALIZACAO'].replace('SEM INFORMACAO', np.nan)
df['CONDICAO_VIA'] = df['CONDICAO_VIA'].replace('SEM INFORMACAO', np.nan)
df['CONSERVACAO_VIA'] = df['CONSERVACAO_VIA'].replace('SEM INFORMACAO', np.nan)
#df['SITUACAO_PLACA'] = df['SITUACAO_PLACA'].replace('SEM INFORMACAO', np.nan)
#df['VELOCIDADE_MAX_VIA'] = df['VELOCIDADE_MAX_VIA'].replace('SEM INFORMACAO', np.nan)
df['PERIODO_DO_DIA'] = df['PERIODO_DO_DIA'].replace('SEM INFORMACAO', np.nan)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [39]:
print(df.isnull().sum()/len(df))

NATUREZA_ACIDENTE    0.007309
TIPO                 0.000983
AUTO                 0.000000
MOTO                 0.000000
CICLOM               0.000000
CICLISTA             0.000000
PEDESTRE             0.000000
ONIBUS               0.000000
CAMINHAO             0.000000
VIATURA              0.000000
TEMPO_CLIMA          0.739553
SITUACAO_SEMAFORO    0.000000
SINALIZACAO          0.744415
CONDICAO_VIA         0.740381
CONSERVACAO_VIA      0.744087
PERIODO_DO_DIA       0.000000
dtype: float64


In [40]:
df = df[df['CONSERVACAO_VIA'].notna()]
df = df[df['SINALIZACAO'].notna()]
df = df[df['CONDICAO_VIA'].notna()]
df = df[df['TEMPO_CLIMA'].notna()]
df = df[df['NATUREZA_ACIDENTE'].notna()]

In [41]:
len(df)

14568

In [42]:
print(df.isnull().sum()/len(df))

NATUREZA_ACIDENTE    0.0
TIPO                 0.0
AUTO                 0.0
MOTO                 0.0
CICLOM               0.0
CICLISTA             0.0
PEDESTRE             0.0
ONIBUS               0.0
CAMINHAO             0.0
VIATURA              0.0
TEMPO_CLIMA          0.0
SITUACAO_SEMAFORO    0.0
SINALIZACAO          0.0
CONDICAO_VIA         0.0
CONSERVACAO_VIA      0.0
PERIODO_DO_DIA       0.0
dtype: float64


In [43]:
df.head()

,NATUREZA_ACIDENTE,TIPO,AUTO,MOTO,CICLOM,CICLISTA,PEDESTRE,ONIBUS,CAMINHAO,VIATURA,TEMPO_CLIMA,SITUACAO_SEMAFORO,SINALIZACAO,CONDICAO_VIA,CONSERVACAO_VIA,PERIODO_DO_DIA
38916,SEM VITIMA,ABALROAMENTO LONGITUDINAL,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,SEM DEFEITO,PERFEITO ESTADO,SECA,PERFEITO ESTADO,MADRUGADA
38919,SEM VITIMA,COLISAO TRASEIRA,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,NAO EXISTE,NAO EXISTENTE,SECA,PERFEITO ESTADO,MANHA
38920,SEM VITIMA,COLISAO TRASEIRA,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,SEM DEFEITO,PERFEITO ESTADO,SECA,PERFEITO ESTADO,MANHA
38921,SEM VITIMA,ABALROAMENTO LONGITUDINAL,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,BOM,NAO EXISTE,PERFEITO ESTADO,SECA,PERFEITO ESTADO,TARDE
38922,SEM VITIMA,CHOQUE VEICULO PARADO,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BOM,NAO EXISTE,NAO EXISTENTE,SECA,PERFEITO ESTADO,TARDE


In [44]:
df['TIPO'].unique()

array(['ABALROAMENTO LONGITUDINAL', 'COLISAO TRASEIRA',
       'CHOQUE VEICULO PARADO', 'ABALROAMENTO TRANSVERSAL', 'CAPOTAMENTO',
       'COLISAO FRONTAL', 'COLISAO COM CICLISTA', 'ATROPELAMENTO',
       'CHOQUE OBJETO FIXO', 'ATROPELAMENTO ANIMAL', 'COLISAO',
       'ACID  DE PERCURSO', 'CHOQUE', 'TOMBAMENTO', 'ENGAVETAMENTO'],
      dtype=object)

In [45]:
df['TEMPO_CLIMA'].unique()

array(['BOM', 'CHUVOSO', 'NUBLADO'], dtype=object)

In [46]:
df['SITUACAO_SEMAFORO'].unique()

array(['SEM DEFEITO', 'NAO EXISTE', 'INTERMITENTE', 'SEM INFORMACAO',
       'COM DEFEITO', 'DESLIGADO'], dtype=object)

In [47]:
df['SINALIZACAO'].unique()

array(['PERFEITO ESTADO', 'NAO EXISTENTE', 'ILEGIVEL', 'INCOMPLETA'],
      dtype=object)

In [48]:
df['CONDICAO_VIA'].unique()

array(['SECA', 'MOLHADA', 'OUTROS', 'OLEOSA'], dtype=object)

In [49]:
df['CONSERVACAO_VIA'].unique()

array(['PERFEITO ESTADO', 'MAL ILUMINADA', 'MAL CONSERVADA', 'NAO HA',
       'OUTROS'], dtype=object)

In [50]:
df['PERIODO_DO_DIA'].unique()

array(['MADRUGADA', 'MANHA', 'TARDE', 'NOITE'], dtype=object)

In [51]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
X_df = df.drop('NATUREZA_ACIDENTE', axis=1)
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[0,9,10,11,12,13,14])],remainder='passthrough')
X_df = np.array(ct.fit_transform(X_df))

In [52]:
print(X_df[0])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0.
 0.]


In [53]:
# Dados
from sklearn.model_selection import train_test_split
import seaborn as sns
#X_df = df.drop('NATUREZA_ACIDENTE', axis=1)
y_df = df['NATUREZA_ACIDENTE']
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df, test_size=0.2, random_state=1)

Xtrain, X_val, ytrain, y_val = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2
#Xtrain, Xtest, ytrain, ytest = train_test_split(X_df, y_df,
#                                                random_state=1)

#GAUSSIANNB

In [54]:
from sklearn.naive_bayes import GaussianNB # 1. escolha da técnica
model = GaussianNB()                       # 2. instanciação do modelo

model.fit(Xtrain, ytrain)                  # 3. criação do modelo
y_model = model.predict(Xtest)             # 4. predição em novos dados
ypred_train = model.predict(Xtrain)  

with mlflow.start_run(run_name="Gaussian Naive Bayes - Train"):
     
  from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
  #print("ACC TRAINING:" + str(accuracy_score(ytrain, ypred_train)))
  #print("ACC TEST:" + str(accuracy_score(ytest, y_model)))
  mlflow.log_metric("Accuracy", accuracy_score(ytrain, ypred_train))
  mlflow.log_metric("Precision", precision_score(ytrain, ypred_train, average='macro'))
  mlflow.log_metric("Recall score", recall_score(ytrain, ypred_train, average='macro'))
  mlflow.log_metric("F1 score", f1_score(ytrain, ypred_train,average='macro'))

with mlflow.start_run(run_name="Gaussian Naive Bayes - Test"):
  mlflow.log_metric("Accuracy", accuracy_score(ytest, y_model))
  mlflow.log_metric("Precision", precision_score(ytest, y_model, average='macro'))
  mlflow.log_metric("Recall score", recall_score(ytest, y_model, average='macro'))
  mlflow.log_metric("F1 score", f1_score(ytest, y_model,average='macro'))

#MLP

In [55]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=50, alpha=0.0001, solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
model.fit(Xtrain, ytrain)                 
y_model = model.predict(Xtest)           
ypred_train = model.predict(Xtrain)      

Iteration 1, loss = 0.58818649
Iteration 2, loss = 0.52302285
Iteration 3, loss = 0.50534097
Iteration 4, loss = 0.48867542
Iteration 5, loss = 0.47192654
Iteration 6, loss = 0.45495726
Iteration 7, loss = 0.43817591
Iteration 8, loss = 0.42196119
Iteration 9, loss = 0.40644047
Iteration 10, loss = 0.39219508
Iteration 11, loss = 0.37990464
Iteration 12, loss = 0.36921189
Iteration 13, loss = 0.36059302
Iteration 14, loss = 0.35355778
Iteration 15, loss = 0.34790613
Iteration 16, loss = 0.34338793
Iteration 17, loss = 0.33944440
Iteration 18, loss = 0.33618231
Iteration 19, loss = 0.33343200
Iteration 20, loss = 0.33089463
Iteration 21, loss = 0.32854247
Iteration 22, loss = 0.32648108
Iteration 23, loss = 0.32464116
Iteration 24, loss = 0.32279620
Iteration 25, loss = 0.32111168
Iteration 26, loss = 0.31958925
Iteration 27, loss = 0.31815491
Iteration 28, loss = 0.31675351
Iteration 29, loss = 0.31548677
Iteration 30, loss = 0.31421752
Iteration 31, loss = 0.31313802
Iteration 32, los

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [56]:
with mlflow.start_run(run_name="MLP Classifier - Train"):
     
  from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
  #print("ACC TRAINING:" + str(accuracy_score(ytrain, ypred_train)))
  #print("ACC TEST:" + str(accuracy_score(ytest, y_model)))
  mlflow.log_metric("Accuracy", accuracy_score(ytrain, ypred_train))
  mlflow.log_metric("Precision", precision_score(ytrain, ypred_train, average='macro'))
  mlflow.log_metric("Recall score", recall_score(ytrain, ypred_train, average='macro'))
  mlflow.log_metric("F1 score", f1_score(ytrain, ypred_train,average='macro'))

with mlflow.start_run(run_name="MLP Classifier- Test"):
  mlflow.log_metric("Accuracy", accuracy_score(ytest, y_model))
  mlflow.log_metric("Precision", precision_score(ytest, y_model, average='macro'))
  mlflow.log_metric("Recall score", recall_score(ytest, y_model, average='macro'))
  mlflow.log_metric("F1 score", f1_score(ytest, y_model,average='macro'))

#RandomForest

In [57]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000)
model.fit(Xtrain, ytrain)                 
y_model = model.predict(Xtest)           
ypred_train = model.predict(Xtrain)   

In [58]:
with mlflow.start_run(run_name="Random Forest - Train"):
     
  from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
  #print("ACC TRAINING:" + str(accuracy_score(ytrain, ypred_train)))
  #print("ACC TEST:" + str(accuracy_score(ytest, y_model)))
  mlflow.log_metric("Accuracy", accuracy_score(ytrain, ypred_train))
  mlflow.log_metric("Precision", precision_score(ytrain, ypred_train, average='macro'))
  mlflow.log_metric("Recall score", recall_score(ytrain, ypred_train, average='macro'))
  mlflow.log_metric("F1 score", f1_score(ytrain, ypred_train,average='macro'))

with mlflow.start_run(run_name="Random Forest - Test"):
  mlflow.log_metric("Accuracy", accuracy_score(ytest, y_model))
  mlflow.log_metric("Precision", precision_score(ytest, y_model, average='macro'))
  mlflow.log_metric("Recall score", recall_score(ytest, y_model, average='macro'))
  mlflow.log_metric("F1 score", f1_score(ytest, y_model,average='macro'))

#KNN

In [59]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)
model.fit(Xtrain, ytrain)                 
y_model = model.predict(Xtest)           
ypred_train = model.predict(Xtrain)

In [60]:
with mlflow.start_run(run_name="KNeighbors Classifier - Train"):
     
  from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
  #print("ACC TRAINING:" + str(accuracy_score(ytrain, ypred_train)))
  #print("ACC TEST:" + str(accuracy_score(ytest, y_model)))
  mlflow.log_metric("Accuracy", accuracy_score(ytrain, ypred_train))
  mlflow.log_metric("Precision", precision_score(ytrain, ypred_train, average='macro'))
  mlflow.log_metric("Recall score", recall_score(ytrain, ypred_train, average='macro'))
  mlflow.log_metric("F1 score", f1_score(ytrain, ypred_train,average='macro'))

with mlflow.start_run(run_name="KNeighbors Classifier - Test"):
  mlflow.log_metric("Accuracy", accuracy_score(ytest, y_model))
  mlflow.log_metric("Precision", precision_score(ytest, y_model, average='macro'))
  mlflow.log_metric("Recall score", recall_score(ytest, y_model, average='macro'))
  mlflow.log_metric("F1 score", f1_score(ytest, y_model,average='macro'))

In [61]:
#run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background


# create remote tunnel using ngrok.com to allow local port access
# borrowed from https://colab.research.google.com/github/alfozan/MLflow-GBRT-demo/blob/master/MLflow-GBRT-demo.ipynb#scrollTo=4h3bKHMYUIG6

from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://453c63d62f5c.ngrok.io
